![imagen](./img/ejercicios.png)

# INFORMACIÓN DEL PROYECTO

### TÍTULO

In [ ]:
#Escribe el título del proyecto en Markdown
titulo = "Artistas o grupos musicales con más exitos"

edades de los artistas o grupos top

univariante
decidir que hacer con los géneros
columna por género
    

### TEMA

In [ ]:
# Describe brevemente el tema del proyecto en Markdown
tema = "Exitos musicales desde 1966"

### HIPÓTESIS

Define aquí lo que será la hipótesis de tu proyecto.
Deberás tener más de una, pero tu proyecto lo harás teniendo una principal hipótesis.

In [ ]:
# Escribe tu hipótesis principal en Markdown
hipotesis_1 = "Qué grupo o artista es el que más éxitos ha tenido desde 1966"

In [ ]:
# Escribe más hipótesis si las hubiera en Markdown
hipotesis_2 = "Que grupo o artista llevan más años en las listas más tiempo y de que generos"
hipotesis_3 = "Que grupo o artista llevan más exitos en los ultimos 10 años y de que generos"
hipotesis_4 = "Qué grupo o artista han sido 1 hit wonder y de que generos"
hipotesis_5 = "Qué generos corresponden a cada grupo o artista anterior"

# ...


## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

Incluye aquí una vista del dataset o datasets de los que partirás para poder evaluar tu hipótesis. <br>
También incluye el origen de estos datos y su fuente.

In [24]:
import ast
import json
import numpy as np
import pandas as pd
# guarda en variables los datasets y su fuente
df_Hot_100_Audio_Features = pd.read_csv("./data/Hot_100_Audio_Features.csv").set_index('index')
df_Hot_Stuff = pd.read_csv("./data/Hot_Stuff.csv").set_index('index')

fuente_1 = ""
# df_2 = pd.DataFrame()
# fuente_2 = ""
# df_3 = pd.DataFrame()
# fuente_3 = ""
# df_4 = pd.DataFrame()
# fuente_4 = ""
# df_5 = pd.DataFrame()
# fuente_5 = ""
# ...

https://www.kaggle.com/datasets/thedevastator/billboard-hot-100-audio-features

The Billboard Hot 100 Weekly Charts with Audio dataset is a comprehensive collection that combines the historical data of the Billboard Hot 100 weekly singles charts with detailed audio features extracted from Spotify. The dataset provides valuable insights into the popularity and musical attributes of songs that have appeared on the Billboard charts.

The primary dataset, Hot Stuff.csv, includes information about each song's position on the weekly charts. It contains columns such as the Billboard Chart URL, WeekID, Song name, Performer name, unique SongID (concatenation of song and performer), Current week on chart, Instance (indicating breaks in chart appearances), Previous week position, Peak Position (highest chart position reached), and Weeks on Chart.

The second dataset, Hot 100 Audio Features.csv, provides in-depth audio features of each song sourced from Spotify's Web API. This includes various metrics such as danceability (suitability for dancing based on musical elements), energy level (intensity and activity), key (musical key signature), loudness (overall volume level in decibels dB), mode (major or minor key), speechiness rating (presence of spoken words in songs), acousticness rating (acoustic quality measure), instrumentalness rating (likelihood of a song being instrumental), liveness rating (presence of a live audience during recording/performance) valence rating(musical positiveness conveyed by a song). Additionally it provides tempo in BPM and time signature(e.g., 4/4 -the rhythm pattern).

Furthermore , this comprehensive dataset encompasses Spotify-related features: track preview URL for audio samples before full streaming or purchase decisions; total duration measured in milliseconds; explicit content indication; album details for songs; genre details provided by Spotify.

With this combined data set, researchers can analyze trends and patterns over time regarding how different audio features relate to a song's popularity and performance on the Billboard Hot 100. It offers endless possibilities for studying the influence of specific music attributes on commercial success and understanding the preferences of popular music audiences.

Whether you are interested in exploring genre-based trends, discovering correlations between chart positions and audio features, or investigating how certain attributes contribute to a song's longevity on the charts, this dataset serves as a valuable resource for deep analysis and insights into Billboard Hot 100 songs

Understanding the Datasets:
<pre>
The dataset consists of two files: Hot Stuff.csv and Hot 100 Audio Features.csv.
The Hot Stuff.csv file contains the weekly Hot 100 singles chart data, including song names, performer names, chart positions, 
and other relevant information.
The Hot 100 Audio Features.csv file contains detailed audio features for each song extracted from Spotify, such as danceability, 
energy, instrumentalness, etc.
Both files can be merged using common attributes like Performer and Song to get a combined view of both datasets.
Exploring the Hot Stuff.csv File:
</pre>
<pre>
This file provides information about each song's position on that week's Hot 100 singles chart.
Important columns in this file are:

    WeekID: The week identifier.
    Song name: The name of the song.
    Performer name: The name of the performer or artist.
    Current week on chart: Represents how many weeks the song has been on the chart at that particular point in time.
    Instance: Indicates whether it is a separate entry for an already listed song (for example, an instance value of 6 means it 
            appeared for the sixth time).
    Previous week position: The position of the song on the previous week's chart.
    Peak Position: The highest position reached by a particular song on any given week's chart.
    Weeks on Chart: Represents how many weeks a specific entry has spent on the chart so far.
    
Exploring the Hot 100 Audio Features.csv File:
</pre>
<pre>
This file provides detailed audio features for each song extracted from Spotify using the Spotify Web API.
It contains attributes like danceability, energy, instrumentalness, tempo, etc., which help capture different aspects of the song's musical characteristics.
Important columns in this file are:

    Performer: The name of the performer or artist of the song.
    Song: The name of the song.
    spotify_genre: The genre(s) of the song according to Spotify.
    spotify_track_duration_ms: The duration
</pre>

<pre>
Predicting the popularity of a song: 
    By analyzing the audio features such as danceability, energy, valence, and tempo, this dataset can be used to develop models that 
    predict the popularity of a song on the Billboard Hot 100 chart.
Genre analysis: 
    With the spotify_genre column providing information about the genre(s) of each song according to Spotify, this dataset can be 
    utilized to analyze trends in popular genres over time and identify patterns in genre preferences among listeners.
Impact of explicit content on chart performance: 
    By examining the spotify_track_explicit column and comparing it with other variables like peak position and weeks on chart, this 
    dataset can be used to explore whether songs with explicit content are more or less likely to achieve high positions on the 
    Billboard Hot 100 chart
</pre>

If you use this dataset in your research, please credit the original authors.

Data Source

https://www.kaggle.com/datasets/thedevastator/billboard-hot-100-audio-features#:~:text=the%20original%20authors.-,Data%20Source,-License

You are free to:
Share - copy and redistribute the material in any medium or format for any purpose, even commercially.
Adapt - remix, transform, and build upon the material for any purpose, even commercially.
You must:
Give appropriate credit - Provide a link to the license, and indicate if changes were made.
ShareAlike - You must distribute your contributions under the same license as the original.
Keep intact - all notices that refer to this license, including copyright notices.

In [ ]:
df_Hot_Stuff.describe

Muestra mediante un head() los principales datasets con los que vas a trabajar

In [ ]:
# Primer dataset
df_Hot_100_Audio_Features
#columnas por género en otro df
    #univariante 
#bivariante con los géneros con más canciones
#duración media de la canción
#heatmap tabla contingencia de los géneros
    #en los colores 

In [ ]:
df_Hot_100_Audio_Features.spotify_genre.head(25)
#transformar lista en columnas


In [25]:
#convierto en otra columna los géneros en una lista 
df_Hot_100_Audio_Features_backup = df_Hot_100_Audio_Features.copy()
#los vacíos son ahora del género unknown 
df_Hot_100_Audio_Features.spotify_genre.fillna("['unknown']",inplace=True)
#los nan de deanceability son ahora 0
df_Hot_100_Audio_Features.danceability.fillna(0,inplace=True)
#con este instrucción creo la columna género adicional como lista 
df_Hot_100_Audio_Features["spotify_genre_list"] = df_Hot_100_Audio_Features.spotify_genre.apply(ast.literal_eval)

In [26]:
#reemplazo los espacios en los nombres por un guión bajo para poder referenciarlos correctamente cuando sea el nombre de una columna
for indice, valor in enumerate(df_Hot_100_Audio_Features.spotify_genre_list):
    #print(f"indice:{indice} con valor:{valor}")
    if len(valor)>1:
        lista_temporal=[]
        for elemento in valor:
            elemento = elemento.replace(" ","_")
            lista_temporal.append(elemento)
        df_Hot_100_Audio_Features.spotify_genre_list.at[indice] = lista_temporal



In [27]:
#creo un diccionario para crear las columnnas del dataset de generos
list_with_all_genres =[]
for valor in df_Hot_100_Audio_Features.spotify_genre_list:
    if len(valor)==1:
        if valor[0] not in list_with_all_genres:
            list_with_all_genres.append(valor[0])
    elif len(valor)>1:
        i=0
        for i in range(len(valor)):
            if valor[i] not in list_with_all_genres:
                list_with_all_genres.append(valor[i])
print(f"Longitud de todos los géneros:{len(list_with_all_genres)}")

Longitud de todos los géneros:1384


In [28]:
super_list_genres =[]#para la tabla
super_dict_genres={}#para filtrar el top 50 de géneros
#super_dict_genres_columns={}#para filtrar el top 50 de géneros
#super_dict_genres_columns_SongID={"SongID":None}
for valor in df_Hot_100_Audio_Features.spotify_genre_list:
    if len(valor)==1:
        if valor[0] not in super_list_genres:
            super_list_genres.append(valor[0])
    elif len(valor)>1:
        i=0
        for i in range(len(valor)):
            if valor[i] not in super_list_genres:
                super_list_genres.append(valor[i])

for clave in super_list_genres:#inicializo el diccionario
        super_dict_genres[clave]=0
#        super_dict_genres_columns[clave]=False

#super_dict = {**super_dict_genres_columns, **super_dict_genres_columns_SongID}
#print(super_dict.items())
#lista_temporal_2=list(super_dict.keys())
#print(type(lista_temporal_2))
super_dict_genres


{'unknown': 0,
 'novelty': 0,
 'adult_standards': 0,
 'brill_building_pop': 0,
 'easy_listening': 0,
 'mellow_gold': 0,
 'rock-and-roll': 0,
 'space_age_pop': 0,
 'surf_music': 0,
 'dance_pop': 0,
 'pop': 0,
 'post-teen_pop': 0,
 'east_coast_hip_hop': 0,
 'hip_hop': 0,
 'pop_rap': 0,
 'rap': 0,
 'southern_hip_hop': 0,
 'album_rock': 0,
 'bubblegum_pop': 0,
 'country_rock': 0,
 'folk_rock': 0,
 'new_wave_pop': 0,
 'soft_rock': 0,
 'yacht_rock': 0,
 'english indie rock': 0,
 'country': 0,
 'country_dawn': 0,
 'nashville_sound': 0,
 'funk': 0,
 'motown': 0,
 'neo_soul': 0,
 'new_jack_swing': 0,
 'quiet_storm': 0,
 'r&b': 0,
 'soul': 0,
 'urban_contemporary': 0,
 'australian_country': 0,
 'contemporary_country': 0,
 'country_road': 0,
 'blues_rock': 0,
 'garage_rock': 0,
 'modern_blues_rock': 0,
 'neo-psychedelic': 0,
 'nu_gaze': 0,
 'punk_blues': 0,
 'folk': 0,
 'singer-songwriter': 0,
 'memphis_soul': 0,
 'southern_soul': 0,
 'classic_soul': 0,
 'disco': 0,
 'jazz_funk': 0,
 'lounge': 0,

In [29]:
for valor in df_Hot_100_Audio_Features.spotify_genre_list:#acumulo en la clave del género
    if len(valor)==1:
            super_dict_genres[valor[0]]=super_dict_genres[valor[0]]+1
    elif len(valor)>1:
        i=0
        for i in range(len(valor)):
            super_dict_genres[valor[i]]=super_dict_genres[valor[i]]+1

In [30]:
#super_dict_genres
top_50_entries = dict(sorted(super_dict_genres.items(), key=lambda item: item[1], reverse=True)[:50]) #50 campos porque de lo contrario eran 1384 columnas
valor_de_corte = list(top_50_entries.values())[-1]
valor_de_corte
top_50_entries
list_top_50_entries = list(top_50_entries.keys())
list_top_50_entries

['unknown',
 'mellow_gold',
 'soft_rock',
 'adult_standards',
 'brill_building_pop',
 'rock',
 'dance_pop',
 'pop',
 'soul',
 'motown',
 'pop_rap',
 'rap',
 'folk_rock',
 'album_rock',
 'classic_rock',
 'quiet_storm',
 'hip_hop',
 'rock-and-roll',
 'bubblegum_pop',
 'funk',
 'classic_soul',
 'urban_contemporary',
 'country',
 'r&b',
 'disco',
 'post-teen_pop',
 'new_wave_pop',
 'contemporary_country',
 'country_rock',
 'country_road',
 'southern_soul',
 'trap',
 'folk',
 'pop_rock',
 'southern_hip_hop',
 'hard_rock',
 'lounge',
 'hip_pop',
 'rockabilly',
 'rhythm_and_blues',
 'art_rock',
 'dance_rock',
 'merseybeat',
 'new_jack_swing',
 'easy_listening',
 'doo-wop',
 'classic_uk_pop',
 'psychedelic_rock',
 'gangster_rap',
 'roots_rock']

In [31]:
#list_genres
#for valor in list_genres:
#    print(f"elemento {valor} en list_genres es del tipo {type(valor)} y tiene una longitud de {len(valor)}")
#len(list_genres)

# Define the number of rows and columns
num_rows = len(df_Hot_100_Audio_Features) 
num_columns = len(list_top_50_entries) #50 campos

# Create an empty dictionary to store column data
column_data = {}

# Populate the dictionary with default values for each column
for i in range(num_columns):
    column_name = list_top_50_entries[i]
    default_value = False
    column_data[column_name] = [default_value] * num_rows

# Create the DataFrame using the populated dictionary
df_genres = pd.DataFrame(column_data)

# Display the DataFrame
df_backup = df_genres.copy()
df_genres


,unknown,mellow_gold,soft_rock,adult_standards,brill_building_pop,rock,dance_pop,pop,soul,motown,...,art_rock,dance_rock,merseybeat,new_jack_swing,easy_listening,doo-wop,classic_uk_pop,psychedelic_rock,gangster_rap,roots_rock
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29500,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29501,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [32]:
df_genres[["spotify_genre_list","SongID"]] = df_Hot_100_Audio_Features[["spotify_genre_list","SongID"]]


In [39]:
#df_genres
df_genres_modificado = df_genres.copy()
#df_genres_modificado.to_json("./data/df_genres_modificado.json",column_name=)
# Convert DataFrame to JSON with column names included
df_genres_modificado.to_json('./data/data_with_columns_orient_split.json', orient='split')
#json_data = df_genres_modificado.to_dict(orient='records')

# Save JSON to a file
#with open('./data/data_with_columns_v2.json', 'w') as json_file:
#    json_file.write(str(df_genres_modificado.columns.tolist()) + '\n')
#    for record in json_data:
#        json_file.write(str(record) + '\n')


In [40]:
df_genres_modificado_json = pd.read_json('./data/data_with_columns_orient_split.json', orient='split')


In [41]:
# Compare the DataFrames
comparison = df_genres_modificado.compare(df_genres_modificado_json)

# Display the differences
print("Differences between df_genres_modificado and df_genres_modificado_json:")
print(comparison)

Differences between df_genres_modificado and df_genres_modificado_json:
Empty DataFrame
Columns: []
Index: []


In [45]:
borrados_un_solo_genero=0
borrados_multiples_generos=0
indices_a_borrar=[]
for indice,fila in df_genres.iterrows():
    indice_df_genres_modificado = df_genres_modificado[df_genres_modificado["SongID"]==fila.SongID].index[0]

    if len(fila.spotify_genre_list)==1:
        #print(f"len(fila.spotify_genre_list):{len(fila.spotify_genre_list)}")
        if fila.spotify_genre_list[0] in df_genres_modificado:
            #print(f"fila.SongID:{fila.SongID} fila.spotify_genre_list[0]:{fila.spotify_genre_list[0]}")
            #print(f"indice_df_genres_modificado:{indice_df_genres_modificado}")
            df_genres_modificado.at[indice_df_genres_modificado,fila.spotify_genre_list[0]]=True
            #print(f"df_genres_modificado:{df_genres_modificado[df_genres_modificado['SongID']==fila.SongID]}")
            #df_genres_modificado.loc["SongID"==fila.SongID,fila.spotify_genre_list[0]]=True
    #    print(f"df_genres[df_genres['SongID']==cancion].index:{df_genres[df_genres['SongID']==fila.SongID].index}")
        else:
            #borramos el registro
            #print(f"borro: {fila.SongID} con indice:{indice_df_genres_modificado} y este género:{fila.spotify_genre_list[0]}")
            #df_genres_modificado.drop(indice_df_genres_modificado,inplace=True)
            indices_a_borrar.append(indice_df_genres_modificado)
            borrados_un_solo_genero=borrados_un_solo_genero+1
    else:
        columnas_a_actualizar=[]
        total_generos = len(fila.spotify_genre_list)
        #contador_total_generos = 0
        for elemento in fila.spotify_genre_list:
            if elemento in df_genres_modificado:
                #print(f"fila.SongID:{fila.SongID} fila.spotify_genre_list[0]:{fila.spotify_genre_list[0]}")
                columnas_a_actualizar.append(elemento)
                #print(f"indice_df_genres_modificado:{indice_df_genres_modificado}")
                #print(f"df_genres_modificado:{df_genres_modificado[df_genres_modificado['SongID']==fila.SongID]}")
        if len(columnas_a_actualizar)!=0:
        #print(f"columnas_a_actualizar:{columnas_a_actualizar}")
            df_genres_modificado.loc[indice_df_genres_modificado,columnas_a_actualizar]=True
        else:
            #print(f"borro en el segundo bucle: {fila.SongID} con indice:{indice_df_genres_modificado} y estos géneros:{fila.spotify_genre_list}")
            borrados_multiples_generos=borrados_multiples_generos+1
            #df_genres_modificado.drop(indice_df_genres_modificado,inplace=True)
            indices_a_borrar.append(indice_df_genres_modificado)
df_genres_modificado.drop(indices_a_borrar,inplace=True)
print(f"Se han borrado borrados_un_solo_genero:{borrados_un_solo_genero}")
print(f"Se han borrado borrados_multiples_generos:{borrados_multiples_generos}")

Se han borrado borrados_un_solo_genero:2401
Se han borrado borrados_multiples_generos:2063


In [46]:
#df_Hot_100_Audio_Features.spotify_genre_list = df_Hot_100_Audio_Features.spotify_genre_list.replace(" ","_",inplace=True)
df_genres_modificado.to_json('./data/df_genres_20231226_2110.json', orient='split')

In [59]:
indices_unknown=list(df_genres_modificado.loc[df_genres_modificado["unknown"]==True].index)
df_genres_modificado.drop(indices_unknown,inplace=True)


In [63]:
#df_genres_modificado.drop(columns="unknown",axis=1)
df_genres_modificado
df_genres_modificado.to_json('./data/df_genres_20231226_2122_without_column_unknown.json', orient='split')

In [55]:
df_genres_modificado.loc[[29501],["lounge"]]

,lounge
29501,True


In [ ]:
#df_result = pd.merge(df_Hot_100_Audio_Features, df_Hot_Stuff, how="inner", on=["Song", "SongID"])

In [ ]:
# Siguiente dataset...
# df_2.head()
df_Hot_100_Audio_Features_EDA=df_genres_modificado.copy()
